<a href="https://colab.research.google.com/github/Qen-byte/ML-with-python/blob/main/Cats_dogs_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/Dataset/training_set'
print(os.listdir(path))

['cats', 'dogs']


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values to [0,1]
    rotation_range=20,        # Randomly rotate images
    width_shift_range=0.2,    # Horizontal shift
    height_shift_range=0.2,   # Vertical shift
    shear_range=0.2,          # Shear transformation
    zoom_range=0.2,           # Random zoom
    horizontal_flip=True,     # Flip horizontally
    validation_split=0.2      # 20% of training data for validation
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/training_set',  # Replace with your training folder path
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'         # 80% of the data
)

# Validation generator (split from training dataset)
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/training_set',  # Same training folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'       # 20% of the data
)

# Test generator (separate dataset, no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/test_set',  # Replace with your testing folder path
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False             # For consistent evaluation
)

Found 5932 images belonging to 2 classes.
Found 1483 images belonging to 2 classes.
Found 2006 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 2168s 12s/step - accuracy: 0.5262 - loss: 0.8415 - val_accuracy: 0.5711 - val_loss: 0.6907
Epoch 2/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 857s 5s/step - accuracy: 0.5470 - loss: 0.6887 - val_accuracy: 0.5968 - val_loss: 0.6756
Epoch 3/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 856s 4s/step - accuracy: 0.5912 - loss: 0.6739 - val_accuracy: 0.5421 - val_loss: 0.6891
Epoch 4/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 838s 4s/step - accuracy: 0.5818 - loss: 0.6731 - val_accuracy: 0.6129 - val_loss: 0.6579
Epoch 5/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 862s 5s/step - accuracy: 0.6180 - loss: 0.6546 - val_accuracy: 0.6433 - val_loss: 0.6265
Epoch 6/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 843s 5s/step - accuracy: 0.6394 - loss: 0.6319 - val_accuracy: 0.6716 - val_loss: 0.5930
Epoch 7/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 842s 5s/step - accuracy: 0.6440 - loss: 0.6294 - val_accuracy: 0.6372 - val_loss: 0.6265
Epoch 8/10
186/186 ━━━━━━━━━━━━━━━━━━━━ 850s 5s/step - accuracy: 0.6591 - loss: 0.6209 - val_ac

In [2]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined